In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from time import sleep
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException


from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By


In [ ]:
# lista_de_tablas= []
# df_final= pd.DataFrame()
# for municipio in tqdm(lista_cinco):

#     driver =webdriver.Chrome()
    
#     url_bueno= f"https://www.wunderground.com/weather/es/{municipio}"
    
    
#     driver.get(url_bueno)
#     driver.maximize_window()
#     codigo= []

#     iframe= WebDriverWait(driver, 10).until(EC.presence_of_element_located(('xpath','//*[@id="sp_message_iframe_1165301"]')))       #A la pag q encuentres dale 10 segundos hasta que encuentres el iframe de las cookies

#     driver.switch_to.frame(iframe)

#     sleep(3)
#     driver.find_element("css selector","#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button").click()

#     driver.switch_to.default_content() 

#     sleep(8)
#     driver.find_element("css selector", "#inner-content > div.region-content-top > lib-city-header > div:nth-child(1) > div > div > a.station-name").click()
#     sleep(3)
#     codigomun_sf= driver.find_element("css selector", "#inner-content > div.region-content-top > app-dashboard-header > div.dashboard__header.small-12.ng-star-inserted > div > div.heading > h1").text

#     codigosplit= codigomun_sf.split(" - ")
#     codigo_municipio= codigosplit[1]
    

#     # driver.find_element("css selector", "#modeSelect").click()

#     # driver.find_element("css selector", "#modeSelect > option:nth-child(3)").click()

#     # driver.find_element("css selector", "#main-page-content > div > div > div > lib-history > div.navigation-wrapper > div.selects > button").click()

#     # driver.find_element("css selector", "#history-tab-group > li:nth-child(2) > a").click()
    
#     # tablina= driver.find_element("css selector", "#main-page-content > div > div > div > lib-history > div.history-tabs > lib-history-table > div > div")

#     for mes in tqdm(range(1,11)):
#         url_cambios= f"https://www.wunderground.com/dashboard/pws/{codigo_municipio}/table/2024-01-01/2024-{mes}-01/monthly"
#         driver.get(url_cambios)
#         tablina= driver.find_element("css selector", "#main-page-content > div > div > div > lib-history > div.history-tabs > lib-history-table > div > div").text
#         lista_de_tablas.append(tablina)
        
#     #Hata aquí me lo hace bien    
    
#     for tabla in tqdm(lista_de_tablas):
#         lista=tabla.split("\n")
#         tabla_df=pd.DataFrame(lista, columns= ["tablina"])
#         tabla_df.drop([0,1], inplace= True)
#         tabla_df["tablina"]= tabla_df["tablina"].str.replace("°F", "")
#         tabla_df["tablina"]= tabla_df["tablina"].str.replace("mph", "")
#         tabla_df["tablina"]= tabla_df["tablina"].str.replace("in", "")
#         tabla_df["tablina"]= tabla_df["tablina"].str.replace("%", "")
#         tabla_df["tablina"]= tabla_df["tablina"].str.replace("  ", " ")
        
#         tabla_df = tabla_df["tablina"].str.split(" ",expand=True)
#         df_final=pd.concat([df_final,tabla_df], axis=0)


#     driver.quit()
# driver.close()    
    

sacar todos los nombre, meterlos en la url para que me salga la pag donde están las tablas de cáracteristicas y la imagen

In [2]:
lista_nombres=[]
driver =webdriver.Chrome()
url_inmobiliaria= "https://ehomemadrid.com/"
driver.get(url_inmobiliaria)
driver.maximize_window()

#encontramos el menú "Nuestros Inmuebles"
menu_nuestros_inmuebles = driver.find_element("css selector", "#menu-item-1619 > a")

#simulación sobre menú para desplegar las opciones
actions = ActionChains(driver)
actions.move_to_element(menu_nuestros_inmuebles).perform()
sleep(2)

#pinchamos en todos sus inmuebles
driver.find_element("css selector", "#menu-item-1642 > a").click()

#pinchamos en el nombre
sleep(2)
driver.find_element("css selector", "#main > div.properties-grid > div > div:nth-child(1) > div > div.title > h2 > a").click()

#conseguimos el nombre del inmueble
sleep(3)
nombre_inmueble = driver.find_element("css selector", "#main > h1").text
lista_nombres.append(nombre_inmueble.lower().replace(" ", "-"))

#conseguir la tabla de características
tabla_caracteristicas = driver.find_element("css selector", "#main > div.property-detail > div.pull-left.overview > div > div > table").text

# Cerramos el navegador
driver.close()


In [3]:
lista_nombres #esto si se puede meter en la url


['virgen-de-la-capilla']

In [5]:
#lo convierto en data frame
lista_strings=tabla_caracteristicas.split("\n")
df= pd.DataFrame(lista_strings, columns=["todo"])

#limpio el data frame
df["todo"]= df["todo"].str.replace("#", "")
df["todo"]= df["todo"].str.replace(" €", "")
df["todo"]= df["todo"].str.replace("m2", "")
df =df["todo"].str.split(": ", expand=True)

#lista de encabezados y traspongo
lista_encabezados= ["id","precio","tipo","contrato","localizacion","banios","habitaciones","superficie"]
df= df.T
#cambio los encabezados
df.columns= lista_encabezados

#elimino la primera fila
df.drop([0], inplace= True)

#pongo el formato correcto
df["id"]=df["id"].astype(float)
df["banios"]=df["banios"].astype(float)
df["precio"]=df["precio"].astype(float)
df["habitaciones"]=df["habitaciones"].astype(float)
df["superficie"]=df["superficie"].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 1 to 1
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1 non-null      float64
 1   precio        1 non-null      float64
 2   tipo          1 non-null      object 
 3   contrato      1 non-null      object 
 4   localizacion  1 non-null      object 
 5   banios        1 non-null      float64
 6   habitaciones  1 non-null      float64
 7   superficie    1 non-null      float64
dtypes: float64(5), object(3)
memory usage: 196.0+ bytes


In [23]:
df=pd.DataFrame(lista_strings, columns=["todo"])
df["todo"]= df["todo"].str.split(" ", expand=True)

ValueError: Columns must be same length as key